<p style="text-align: center">
    <img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>
</p>

# Google Play Store - Get sentiment analysis on app reviews using Obsei

<p style="text-align: center">
    <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Play%20Store/Google_Play_Store_reviews_sentiment_analysis_with_Obsei.ipynb" target="_parent">
        <img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
    </a>
</p>

<p style="text-align: center">
    <a href="https://github.com/obsei/obsei">
        <img width="5%" alt="Obsei" src="https://raw.githubusercontent.com/obsei/obsei-resources/master/logos/obsei_200x200.png?w=50"/>
    </a>
</p>

<p style="text-align: center">
    <img width="3%" alt="PlayStore" src="https://raw.githubusercontent.com/obsei/obsei-resources/master/logos/playstore.png"/>
    <img width="5.5%" alt="Arraow" src="https://freepngimg.com/thumb/direction/21565-6-green-arrow-hd.png"/>
    <img width="3%" alt="Sentiment" src="https://raw.githubusercontent.com/obsei/obsei-resources/master/logos/sentiment.png"/>
    <img width="5.5%" alt="Arraow" src="https://freepngimg.com/thumb/direction/21565-6-green-arrow-hd.png"/>
    <img width="3%" alt="Pandas" src="https://raw.githubusercontent.com/obsei/obsei-resources/master/logos/pandas.svg"/>
</p>

## Input

### Pip install Obsei

In [ ]:
!pip install git+git://github.com/obsei/obsei@master#egg=obsei

### Import library

In [ ]:
# Google Play Store Observer imports
from obsei.source.playstore_scrapper import (
    PlayStoreScrapperConfig,
    PlayStoreScrapperSource,
)

# Sentiment Analyzer imports
from obsei.analyzer.sentiment_analyzer import (
    TransformersSentimentAnalyzerConfig,
    TransformersSentimentAnalyzer,
)

# Pandas Informer imports
from obsei.sink.pandas_sink import (
    PandasSink,
    PandasSinkConfig,
)

### Variables

In [ ]:
# Variables for Google Play Store Observer
app_name = "Gmail"
app_url = 'https://play.google.com/store/apps/details?id=com.google.android.gm&hl=en_IN&gl=US' # This is app url of Gmail App
max_reviews = 10

# Variable for Sentiment Analyzer
sentiment_model = "typeform/mobilebert-uncased-mnli" # Refer link for supported models: https://huggingface.co/models?filter=zero-shot-classification

# Variables for Pandas Informer
app_name_col = "app name"
included_cols = []
included_cols.append("segmented_data_classifier_data_positive")
included_cols.append("segmented_data_classifier_data_negative")
included_cols.append("processed_text")
included_cols.append("meta_at")
included_cols.append("meta_score")

rename_cols_dict = {}
rename_cols_dict["segmented_data_classifier_data_positive"] = "positive"
rename_cols_dict["segmented_data_classifier_data_negative"] = "negative"
rename_cols_dict["processed_text"] = "review"
rename_cols_dict["meta_at"] = "time"
rename_cols_dict["meta_score"] = "rating"

cols_order_list = [app_name_col, "time", "rating", "review", "positive", "negative"]

## Model

### Initialize Observer, Analyzer and Informer

In [ ]:
# Initialize Google Play Store Observer and it's config
play_observer_config = PlayStoreScrapperConfig(
    app_url=app_url,
    max_count=max_reviews,
)
play_observer = PlayStoreScrapperSource()

# Initialize Sentiment Analyzer and it's config
sentiment_analyzer_config=TransformersSentimentAnalyzerConfig()

sentiment_analyzer = TransformersSentimentAnalyzer(
   model_name_or_path=sentiment_model
)

# Initialize Pandas Informer and it's config
pandas_informer_config = PandasSinkConfig(
   include_columns_list=included_cols
)
pandas_informer = PandasSink()

### Execute Obsei Workflow

In [ ]:
def execute_obsei_workflow(
    observer,
    observer_config,
    analyzer,
    analyzer_config,
    informer,
    informer_config
):
    observer_responses = observer.lookup(observer_config)
    analyzer_responses = analyzer.analyze_input(observer_responses, analyzer_config)
    dataframe = informer.send_data(analyzer_responses, informer_config)
    return dataframe

### Make DataFrame more presentable

In [ ]:
def beautify_dataframe(
    dataframe,
    rename_cols,
    name,
    name_col,
    order_list
):
    dataframe.rename(rename_cols, axis=1, inplace=True)
    dataframe[name_col] = name
    return dataframe[order_list]

## Output

In [ ]:
df = execute_obsei_workflow(
    play_observer,
    play_observer_config,
    sentiment_analyzer,
    sentiment_analyzer_config,
    pandas_informer,
    pandas_informer_config
)

df = beautify_dataframe(
    df,
    rename_cols_dict,
    app_name,
    app_name_col,
    cols_order_list
)

df